In [16]:
import numpy as np

In [63]:
url = "https://en.wikipedia.org/wiki/The_Refinitiv_Business_Classification"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

wiki_table = soup.find('table', class_="wikitable")

wiki_table_rows = wiki_table.find_all('tr')

industry_group = []
hierarchical_id = []
economic_sector=[]
business_sector=[]
industry=[]
activity=[]


for row in wiki_table_rows:
    cells = row.find_all('td')
    if len(cells) >= 3:
        #if len(cells[3].text) > 1:
            economic_sector.append(cells[0].text.strip('\n'))
            business_sector.append(cells[1].text.strip('\n'))
            industry_group.append(cells[2].text.strip('\n'))
            industry.append(cells[3].text.strip('\n'))
            activity.append(cells[4].text.strip('\n'))
            hierarchical_id.append(cells[5].text.strip('\n'))

results = pd.DataFrame({'economic_sector':economic_sector,
                        'business_sector':business_sector,
                        'industry_group':industry_group,
                        'industry':industry,
                        'activity':activity,
                        'hierarchical_id':hierarchical_id,
})

results

,economic_sector,business_sector,industry_group,industry,activity,hierarchical_id
0,Energy,Energy - Fossil Fuels,Coal,Coal,Coal (NEC),5010101010
1,,,,,Coal Mining Support,5010101011
2,,,,,Coal Wholesale,5010101012
3,,,Oil & Gas,Integrated Oil & Gas,Integrated Oil & Gas,5010201010
4,,,,Oil & Gas Exploration and Production,Oil & Gas Exploration and Production (NEC),5010202010
...,...,...,...,...,...,...
890,,,,,Elementary & Primary Schools,6310201030
891,,,,,Colleges & Secondary Schools,6310201040
892,,,,,Universities,6310201050
893,,,,,School Districts,6310201060


In [64]:
results.economic_sector = results.economic_sector[results.economic_sector.str.strip() != '']

In [65]:
columns=['economic_sector', 'business_sector', 'industry_group', 'industry',
       'activity', 'hierarchical_id']

In [66]:
for column in columns:
    results[column] = results[column][results[column].str.strip() != '']

In [67]:
results

,economic_sector,business_sector,industry_group,industry,activity,hierarchical_id
0,Energy,Energy - Fossil Fuels,Coal,Coal,Coal (NEC),5010101010
1,NaN,NaN,NaN,NaN,Coal Mining Support,5010101011
2,NaN,NaN,NaN,NaN,Coal Wholesale,5010101012
3,NaN,NaN,Oil & Gas,Integrated Oil & Gas,Integrated Oil & Gas,5010201010
4,NaN,NaN,NaN,Oil & Gas Exploration and Production,Oil & Gas Exploration and Production (NEC),5010202010
...,...,...,...,...,...,...
890,NaN,NaN,NaN,NaN,Elementary & Primary Schools,6310201030
891,NaN,NaN,NaN,NaN,Colleges & Secondary Schools,6310201040
892,NaN,NaN,NaN,NaN,Universities,6310201050
893,NaN,NaN,NaN,NaN,School Districts,6310201060


In [68]:
for column in columns:
    results[column] = results[column].ffill()

In [69]:
results.drop(['activity'], axis=1)

,economic_sector,business_sector,industry_group,industry,hierarchical_id
0,Energy,Energy - Fossil Fuels,Coal,Coal,5010101010
1,Energy,Energy - Fossil Fuels,Coal,Coal,5010101011
2,Energy,Energy - Fossil Fuels,Coal,Coal,5010101012
3,Energy,Energy - Fossil Fuels,Oil & Gas,Integrated Oil & Gas,5010201010
4,Energy,Energy - Fossil Fuels,Oil & Gas,Oil & Gas Exploration and Production,5010202010
...,...,...,...,...,...
890,Academic & Educational Services,Academic & Educational Services,"School, College & University","School, College & University",6310201030
891,Academic & Educational Services,Academic & Educational Services,"School, College & University","School, College & University",6310201040
892,Academic & Educational Services,Academic & Educational Services,"School, College & University","School, College & University",6310201050
893,Academic & Educational Services,Academic & Educational Services,"School, College & University","School, College & University",6310201060


In [70]:
results['hierarchical_id']= results['hierarchical_id'].astype(str).str[:-2].astype(np.int64)

In [71]:
results['hierarchical_id']= results['hierarchical_id'].apply(str)

In [97]:
results.dtypes

economic_sector    object
business_sector    object
industry_group     object
industry           object
activity           object
hierarchical_id    object
dtype: object

In [92]:
def get_info_trbc(hierarchical_id):
    str_id= str(hierarchical_id)
    try:
        if len(str_id)==2:
            s = results.loc[results['hierarchical_id'].str.startswith(str_id), 'economic_sector'].reset_index(drop=True)
            return s[0]
        elif len(str_id)==4:
            r = results.loc[results['hierarchical_id'].str.startswith(str_id), 'business_sector'].reset_index(drop=True)
            return r[0]
        elif len(str_id)==6:
            q = results.loc[results['hierarchical_id'].str.startswith(str_id), 'industry_group'].reset_index(drop=True)
            return q[0] 
        elif len(str_id)==8:
            p = results.loc[results['hierarchical_id'].str.startswith(str_id), 'industry'].reset_index(drop=True)
            return p[0] 
        else:
            print("Please enter a valid hierarchical ID.")
    except:
        print("Please enter a valid hierarchical ID.")

In [96]:
get_info_trbc(63102010)

'School, College & University'

In [98]:
results.to_pickle('../MA_PREDICTOR/data/declassification.pkl')
print('Saved as declassification.pkl')

Saved as declassification.pkl
